# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [49]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [50]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [51]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [52]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [53]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [54]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_lib
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [55]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('music_lib')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## query 1
Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [56]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS song_info_session "
query = query + "(sessionId int, itemInSession int, artist text, song_title text, song_length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [57]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_info_session (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [58]:
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song_title, song_length from song_info_session WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# use pretty table to display data in tabular form and include headings
t = PrettyTable(['Artist', 'Song', 'SongLength'])
for row in rows:
    t.add_row([row.artist, row.song_title, row.song_length])
print(t)

+-----------+---------------------------------+--------------------+
|   Artist  |               Song              |     SongLength     |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## query 2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [42]:
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + "(userId int, sessionId int, itemInSession int, artist text, songTitle text, firstName text, lastName text, PRIMARY KEY (userId, sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)         
    
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_playlist_session (userId, sessionId, itemInSession, artist, songTitle, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [43]:
## select from song_playlist_session
query = "SELECT artist, songtitle, firstName, lastName from song_playlist_session WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# use pretty table to display data in tabular form and include headings
t = PrettyTable(['Artist', 'Song', 'FirstName', 'LastName'])
for row in rows:
    t.add_row([row.artist, row.songtitle, row.firstname, row.lastname])
print(t)

+-------------------+------------------------------------------------------+-----------+----------+
|       Artist      |                         Song                         | FirstName | LastName |
+-------------------+------------------------------------------------------+-----------+----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |
+-------------------+------------------------------------------------------+-----------+----------+


## query 3


In [65]:
## listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS user_song "
query = query + "(song text, firstName text, lastName text, PRIMARY KEY (song, firstName, lastName))"
try:
    session.execute(query)
except Exception as e:
    print(e)         
    
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_song (song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], line[1], line[4]))

In [66]:
## select from user_song
query = "SELECT firstName, lastName, song from user_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# use pretty table to display data in tabular form and include headings
t = PrettyTable(['FirstName', 'LastName'])
for row in rows:
    t.add_row([row.firstname, row.lastname])
print(t)

+------------+----------+
| FirstName  | LastName |
+------------+----------+
| Jacqueline |  Lynch   |
|    Sara    | Johnson  |
|   Tegan    |  Levine  |
+------------+----------+


### Drop the tables before closing out the sessions

In [46]:
## Drop the table before closing out the sessions
query = "drop table song_info_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [47]:
session.shutdown()
cluster.shutdown()